<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [2]:
""" Steps, process
1. Get data into dataframe
2. Tokenize data in dataframe
3. chug tokens into gensim corpus?

"""

' Steps, process\n1. Get data into dataframe\n2. Tokenize data in dataframe\n3. chug tokens into gensim corpus?\n\n'

In [3]:
%pwd

'/Users/curtismckendrick/lambda school repositories/Class Notes/Unit-4-New/DS-Unit-4-Sprint-1-NLP/module4-topic-modeling'

In [4]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [5]:
import pandas as pd
import zipfile

path_for_later = './data/Amazon'

path = '/Users/curtismckendrick/lambda school repositories/Class Notes/Unit-4-New/DS-Unit-4-Sprint-1-NLP/module4-topic-modeling'

zf = zipfile.ZipFile(path + '/data/Amazon/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip') 
df = pd.read_csv(zf.open('Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv'))

In [6]:
df.head(2)

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."


In [7]:
STOPWORDS = set(STOPWORDS)

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

def tokenize_df_column(df_col):
    list_of_lists_of_tokens = []
    for string in df_col:
        list_of_tokens = [token for token in simple_preprocess(string) if token not in STOPWORDS]
        list_of_lists_of_tokens.append(list_of_tokens)
    return(list_of_lists_of_tokens)

tokenize("Hello World! This a test of the tokenization method")

['hello', 'world', 'test', 'tokenization', 'method']

In [8]:
rev = pd.DataFrame(index=df['reviews.title'])
rev.head()

""
reviews.title
... 3 of them and one of the item is bad quality. Is missing backup spring so I have ...
... always the less expensive way to go for products like
... are not Duracell but for the price i am happy.
... as well as name brand batteries at a much better
... batteries are very long lasting the price is great.


In [9]:
df['reviews.text'].head()

0    I order 3 of them and one of the item is bad q...
1    Bulk is always the less expensive way to go fo...
2    Well they are not Duracell but for the price i...
3    Seem to work as well as name brand batteries a...
4    These batteries are very long lasting the pric...
Name: reviews.text, dtype: object

In [10]:
# does the same thing as the next cell

rev['tokens'] = tokenize_df_column(df['reviews.text'])
rev.head()

,tokens
reviews.title,
... 3 of them and one of the item is bad quality. Is missing backup spring so I have ...,"[order, item, bad, quality, missing, backup, s..."
... always the less expensive way to go for products like,"[bulk, expensive, way, products, like]"
... are not Duracell but for the price i am happy.,"[duracell, price, happy]"
... as well as name brand batteries at a much better,"[work, brand, batteries, better, price]"
... batteries are very long lasting the price is great.,"[batteries, long, lasting, price, great]"


In [11]:
# more standard say of doing the previous cell

rev['tokens'] = df['reviews.text'].apply(lambda x: tokenize(x)).tolist()
rev.head()

,tokens
reviews.title,
... 3 of them and one of the item is bad quality. Is missing backup spring so I have ...,"[order, item, bad, quality, missing, backup, s..."
... always the less expensive way to go for products like,"[bulk, expensive, way, products, like]"
... are not Duracell but for the price i am happy.,"[duracell, price, happy]"
... as well as name brand batteries at a much better,"[work, brand, batteries, better, price]"
... batteries are very long lasting the price is great.,"[batteries, long, lasting, price, great]"


In [12]:
id2token = corpora.Dictionary(rev['tokens'].tolist())
id2token

In [13]:
len(id2token.keys())

9622

In [14]:
id2token.filter_extremes(no_below=10, no_above=.75)
len(id2token.keys())

2359

In [15]:
corpus = [id2token.doc2bow(text) for text in rev['tokens'].tolist()]

In [17]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2token,
                   random_state=2019,
                   num_topics=15,
                   passes=10,
                   workers=4
                  )

In [ ]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, id2token)

In [105]:
def doc_stream(path):
    for f in os.listdir(path):
        with open(os.path.join(path,f)) as t:
            text = t.read().strip('\n')
            yield text

In [109]:
from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, token_df_column, limit, start=2, step=3, passes=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    path : path to input texts
    limit : Max num of topics
    passes: the number of times the entire lda model & coherence values are calculated

    Returns:
    -------
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    coherence_values = []
    
    tokens = token_df_column.tolist()
    
    for iter_ in range(passes):
        for num_topics in range(start, limit, step):
#             stream = doc_stream(path)
            model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=4)
            coherencemodel = CoherenceModel(model=model, dictionary=dictionary, corpus=corpus, coherence='u_mass')
            coherence_values.append({'pass': iter_, 
                                     'num_topics': num_topics, 
                                     'coherence_score': coherencemodel.get_coherence()
                                    })

    return coherence_values

In [1]:
# # Can take a long time to run.
# coherence_values = compute_coherence_values(dictionary=id2token, 
#                                             corpus=corpus, 
#                                             token_df_column=rev['tokens'], 
#                                             start=2, 
#                                             limit=40, 
#                                             step=6,
#                                             passes=40)

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling